In [51]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score, roc_auc_score

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.naive_bayes import GaussianNB

from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import VotingClassifier

In [52]:
images = pd.read_csv("C:\Hogwarts\machine_learning\Cases\Image Segmentation\Image_Segmentation.csv")

In [53]:
images.head()

,Class,region.centroid.col,region.centroid.row,region.pixel.count,short.line.density.5,short.line.density.2,vedge.mean,vegde.sd,hedge.mean,hedge.sd,intensity.mean,rawred.mean,rawblue.mean,rawgreen.mean,exred.mean,exblue.mean,exgreen.mean,value.mean,saturation.mean,hue-mean
0,BRICKFACE,188,133,9,0.0,0.0,0.333333,0.266667,0.500000,0.077778,6.666666,8.333334,7.777778,3.888889,5.000000,3.333333,-8.333333,8.444445,0.538580,-0.924817
1,BRICKFACE,105,139,9,0.0,0.0,0.277778,0.107407,0.833333,0.522222,6.111111,7.555555,7.222222,3.555556,4.333334,3.333333,-7.666666,7.555555,0.532628,-0.965946
2,BRICKFACE,34,137,9,0.0,0.0,0.500000,0.166667,1.111111,0.474074,5.851852,7.777778,6.444445,3.333333,5.777778,1.777778,-7.555555,7.777778,0.573633,-0.744272
3,BRICKFACE,39,111,9,0.0,0.0,0.722222,0.374074,0.888889,0.429629,6.037037,7.000000,7.666666,3.444444,2.888889,4.888889,-7.777778,7.888889,0.562919,-1.175773
4,BRICKFACE,16,128,9,0.0,0.0,0.500000,0.077778,0.666667,0.311111,5.555555,6.888889,6.666666,3.111111,4.000000,3.333333,-7.333334,7.111111,0.561508,-0.985811


In [54]:
y = images["Class"]

X = images.drop("Class", axis = 1)

In [55]:
y.value_counts()

SKY          30
FOLIAGE      30
CEMENT       30
WINDOW       30
PATH         30
GRASS        30
BRICKFACE    29
Name: Class, dtype: int64

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    stratify = y,
                                                    random_state = 2022,
                                                    train_size = 0.7)

In [57]:
lr = LogisticRegression()

In [58]:
nb = GaussianNB()

In [59]:
params = {"LR__C": np.linspace(0.001, 5, 5),
         "SVML__SVM__C": np.linspace(0.001, 5, 5)}

In [60]:
scaler = StandardScaler()

svm = SVC(probability = True,
         random_state = 2022,
         kernel = "linear")

pipe = Pipeline([("STD", scaler), ("SVM", svm)])

In [61]:
kfold = StratifiedKFold(n_splits = 5,
                        shuffle = True,
                        random_state = 2022)

In [62]:
da = LinearDiscriminantAnalysis()

In [63]:
voting = VotingClassifier([("Logistic Regression", lr),
                          ("NB", nb),
                          ("LDA", da),
                          ("SVML", pipe)],
                         voting = "soft")

In [69]:
print(voting.get_params())

{'estimators': [('Logistic Regression', LogisticRegression()), ('NB', GaussianNB()), ('LDA', LinearDiscriminantAnalysis()), ('SVML', Pipeline(steps=[('STD', StandardScaler()),
                ('SVM',
                 SVC(kernel='linear', probability=True, random_state=2022))]))], 'flatten_transform': True, 'n_jobs': None, 'verbose': False, 'voting': 'soft', 'weights': None, 'Logistic Regression': LogisticRegression(), 'NB': GaussianNB(), 'LDA': LinearDiscriminantAnalysis(), 'SVML': Pipeline(steps=[('STD', StandardScaler()),
                ('SVM',
                 SVC(kernel='linear', probability=True, random_state=2022))]), 'Logistic Regression__C': 1.0, 'Logistic Regression__class_weight': None, 'Logistic Regression__dual': False, 'Logistic Regression__fit_intercept': True, 'Logistic Regression__intercept_scaling': 1, 'Logistic Regression__l1_ratio': None, 'Logistic Regression__max_iter': 100, 'Logistic Regression__multi_class': 'auto', 'Logistic Regression__n_jobs': None, 'Logistic 

In [65]:
gcv = GridSearchCV(voting,
                  param_grid = params,
                  cv = kfold, verbose = 3,
                  scoring = "neg_log_loss")

gcv.fit(X, y)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


ValueError: Invalid parameter LR for estimator VotingClassifier(estimators=[('Logistic Regression', LogisticRegression()),
                             ('NB', GaussianNB()),
                             ('LDA', LinearDiscriminantAnalysis()),
                             ('SVML',
                              Pipeline(steps=[('STD', StandardScaler()),
                                              ('SVM',
                                               SVC(kernel='linear',
                                                   probability=True,
                                                   random_state=2022))]))],
                 voting='soft'). Check the list of available parameters with `estimator.get_params().keys()`.

In [66]:
pd_cv = pd.DataFrame( gcv.cv_results_ )
print(gcv.best_params_)
print(gcv.best_score_)

AttributeError: 'GridSearchCV' object has no attribute 'cv_results_'